Задание 1. Анализ производительности CPU-параллельной программы (OpenMP)

In [6]:
%%writefile task1.cpp

#include <iostream>   // ввод-вывод
#include <vector>     // контейнер std::vector
#include <omp.h>      // OpenMP

int main() {

    // Размер массива
    const int N = 10'000'000;

    // Массив данных, инициализированный значениями 1.0
    std::vector<double> data(N, 1.0);

    // Переменные для измерения времени выполнения
    double start, end;

    // ==============================
    // Начало измерения времени
    // ==============================
    start = omp_get_wtime();

    // ------------------------------
    // Параллельное вычисление суммы
    // ------------------------------
    double sum = 0.0;

    // reduction(+:sum) — каждая нить считает свою локальную сумму,
    // затем все частичные результаты суммируются
    #pragma omp parallel for reduction(+:sum)
    for (int i = 0; i < N; i++) {
        sum += data[i];
    }

    // Вычисление среднего значения
    double mean = sum / N;

    // -------------------------------------
    // Параллельное вычисление дисперсии
    // -------------------------------------
    double variance = 0.0;

    // Аналогично сумме используется редукция
    #pragma omp parallel for reduction(+:variance)
    for (int i = 0; i < N; i++) {
        double diff = data[i] - mean;
        variance += diff * diff;
    }

    // Нормализация дисперсии
    variance /= N;

    // ==============================
    // Окончание измерения времени
    // ==============================
    end = omp_get_wtime();

    // Вывод времени выполнения и результатов вычислений
    std::cout << "Time: " << end - start << " seconds\n";
    std::cout << "Mean: " << mean << "\n";
    std::cout << "Variance: " << variance << "\n";

    return 0;
}


Overwriting task1.cpp


In [7]:
!g++ -fopenmp task1.cpp -o task1
!OMP_NUM_THREADS=1 ./task1
!OMP_NUM_THREADS=2 ./task1
!OMP_NUM_THREADS=4 ./task1
!OMP_NUM_THREADS=8 ./task1

Time: 0.0864647 seconds
Mean: 1
Variance: 0
Time: 0.07834 seconds
Mean: 1
Variance: 0
Time: 0.084849 seconds
Mean: 1
Variance: 0
Time: 0.0859424 seconds
Mean: 1
Variance: 0


Задание 2. Оптимизация доступа к памяти на GPU (CUDA)

In [1]:
%%writefile task2.cu

#include <cuda_runtime.h>
#include <iostream>
#include <vector>

#define N (1 << 24)        // ~16 млн элементов
#define THREADS 256        // число потоков в блоке
#define STRIDE 32          // шаг для некоалесцированного доступа

// =====================================================
// Ядро 1: Коалесцированный доступ к глобальной памяти
// Каждый поток работает с соседним элементом массива
// =====================================================
__global__ void coalesced_kernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] *= 2.0f;
    }
}

// =====================================================
// Ядро 2: Некоалесцированный доступ к памяти
// Потоки обращаются к данным с большим шагом (stride)
// =====================================================
__global__ void noncoalesced_kernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int access_idx = idx * STRIDE;
    if (access_idx < n) {
        data[access_idx] *= 2.0f;
    }
}

// =====================================================
// Ядро 3: Использование разделяемой памяти
// Данные сначала копируются в shared memory,
// затем обрабатываются и записываются обратно
// =====================================================
__global__ void shared_memory_kernel(float* data, int n) {
    __shared__ float buffer[THREADS];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + tid;

    // Загрузка из глобальной памяти в shared memory
    if (idx < n) {
        buffer[tid] = data[idx];
    }

    __syncthreads();

    // Обработка данных в shared memory
    if (idx < n) {
        buffer[tid] *= 2.0f;
    }

    __syncthreads();

    // Запись обратно в глобальную память
    if (idx < n) {
        data[idx] = buffer[tid];
    }
}

// =====================================================
// Функция измерения времени выполнения ядра
// =====================================================
void measure_kernel(
    void (*kernel)(float*, int),
    float* d_data,
    int n,
    const char* label
) {
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int blocks = (n + THREADS - 1) / THREADS;

    cudaEventRecord(start);
    kernel<<<blocks, THREADS>>>(d_data, n);
    cudaEventRecord(stop);

    cudaEventSynchronize(stop);

    float time_ms = 0.0f;
    cudaEventElapsedTime(&time_ms, start, stop);

    std::cout << label << " time: " << time_ms << " ms\n";

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

// =====================================================
// Главная функция
// =====================================================
int main() {

    size_t size = N * sizeof(float);

    // Выделение памяти на CPU
    std::vector<float> h_data(N, 1.0f);

    // Выделение памяти на GPU
    float* d_data;
    cudaMalloc(&d_data, size);

    // Копирование данных на GPU
    cudaMemcpy(d_data, h_data.data(), size, cudaMemcpyHostToDevice);

    std::cout << "CUDA memory access analysis\n";

    // ===== Коалесцированный доступ =====
    measure_kernel(coalesced_kernel, d_data, N,
                   "Coalesced access");

    // ===== Некоалесцированный доступ =====
    measure_kernel(noncoalesced_kernel, d_data, N,
                   "Non-coalesced access");

    // ===== Shared memory =====
    measure_kernel(shared_memory_kernel, d_data, N,
                   "Shared memory");

    // Освобождение памяти GPU
    cudaFree(d_data);

    return 0;
}


Writing task2.cu


In [2]:
!nvcc -arch=sm_75 -gencode=arch=compute_75,code=sm_75 task2.cu -o task2
!./task2

CUDA memory access analysis
Coalesced access time: 0.705184 ms
Non-coalesced access time: 0.655232 ms
Shared memory time: 0.698336 ms


Задание 3. Профилирование гибридного приложения CPU + GPU

In [3]:
%%writefile hybrid_async.cu

#include <iostream>        // ввод-вывод
#include <vector>          // контейнер std::vector
#include <omp.h>           // OpenMP
#include <cuda_runtime.h>  // CUDA Runtime API

#define N 10000000         // размер массива
#define THREADS 256        // количество потоков в CUDA-блоке

// =====================================================
// CUDA-ядро для обработки части массива на GPU
// Каждый поток обрабатывает один элемент
// =====================================================
__global__ void gpu_kernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] *= 2.0f;
    }
}

int main() {

    // =====================================================
    // Подготовка данных на CPU
    // =====================================================
    // Инициализация массива значениями 1.0
    std::vector<float> h_data(N, 1.0f);

    // Делим массив на две части:
    // первая половина — CPU, вторая — GPU
    int half = N / 2;
    size_t size_half = half * sizeof(float);

    // =====================================================
    // Выделение памяти на GPU
    // =====================================================
    float* d_data;
    cudaMalloc(&d_data, size_half);

    // Создание CUDA stream для асинхронных операций
    cudaStream_t stream;
    cudaStreamCreate(&stream);

    // =====================================================
    // Начало измерения времени
    // =====================================================
    double start = omp_get_wtime();

    // =====================================================
    // Асинхронная передача данных CPU → GPU
    // Копируется вторая половина массива
    // =====================================================
    cudaMemcpyAsync(
        d_data,                     // память GPU
        h_data.data() + half,       // вторая половина массива CPU
        size_half,
        cudaMemcpyHostToDevice,
        stream                      // асинхронный stream
    );

    // =====================================================
    // Параллельная обработка CPU и GPU
    // =====================================================
    #pragma omp parallel sections
    {
        // -----------------------------
        // CPU-часть вычислений
        // -----------------------------
        #pragma omp section
        {
            for (int i = 0; i < half; i++) {
                h_data[i] *= 2.0f;
            }
        }

        // -----------------------------
        // GPU-часть вычислений
        // -----------------------------
        #pragma omp section
        {
            int blocks = (half + THREADS - 1) / THREADS;
            gpu_kernel<<<blocks, THREADS, 0, stream>>>(d_data, half);
        }
    }

    // =====================================================
    // Асинхронная передача данных GPU → CPU
    // =====================================================
    cudaMemcpyAsync(
        h_data.data() + half,   // куда копируем
        d_data,                 // откуда копируем
        size_half,
        cudaMemcpyDeviceToHost,
        stream
    );

    // Ожидание завершения всех операций в stream
    cudaStreamSynchronize(stream);

    // =====================================================
    // Окончание измерения времени
    // =====================================================
    double end = omp_get_wtime();

    // =====================================================
    // Освобождение ресурсов
    // =====================================================
    cudaFree(d_data);
    cudaStreamDestroy(stream);

    // Вывод времени выполнения гибридного алгоритма
    std::cout << "Hybrid async time: " << end - start << " seconds\n";

    return 0;
}


Writing hybrid_async.cu


In [5]:
!nvcc -Xcompiler -fopenmp hybrid_async.cu -o hybrid_async
!./hybrid_async

Hybrid async time: 0.0836766 seconds


Задание 4. Анализ масштабируемости распределённой программы (MPI)

In [4]:
%%writefile task4.cpp

#include <mpi.h>        // MPI библиотека
#include <iostream>    // ввод-вывод
#include <vector>      // контейнер std::vector

int main(int argc, char* argv[]) {

    // Инициализация MPI
    MPI_Init(&argc, &argv);

    int rank, size;

    // Номер текущего процесса
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    // Общее количество процессов
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // =================================================
    // Параметры задачи
    // =================================================

    // Размер массива на ОДИН процесс (используется для weak scaling)
    const long long LOCAL_N = 10'000'000;

    // Общий размер массива для strong scaling
    const long long GLOBAL_N = LOCAL_N;

    // -------------------------------------------------
    // Выбор режима масштабирования
    // true  — weak scaling  (нагрузка на процесс фиксирована)
    // false — strong scaling (общий объём данных фиксирован)
    // -------------------------------------------------
    bool weak_scaling = true;

    // Определяем размер локального массива
    long long local_n;
    if (weak_scaling) {
        // Weak scaling: каждый процесс обрабатывает одинаковый объём данных
        local_n = LOCAL_N;
    } else {
        // Strong scaling: общий объём данных делится между процессами
        local_n = GLOBAL_N / size;
    }

    // =================================================
    // Локальные данные
    // =================================================
    // Каждый процесс создаёт свой локальный массив
    std::vector<double> local_data(local_n, 1.0);

    // =================================================
    // Начало измерения времени
    // =================================================
    double start = MPI_Wtime();

    // =================================================
    // Локальные вычисления
    // =================================================
    // Каждый процесс вычисляет сумму своей части массива
    double local_sum = 0.0;
    for (long long i = 0; i < local_n; i++) {
        local_sum += local_data[i];
    }

    // Переменная для глобальной суммы
    double global_sum = 0.0;

    // =================================================
    // Коллективная операция MPI_Reduce
    // =================================================
    // Локальные суммы объединяются в одну глобальную
    // Результат доступен только на процессе rank = 0
    MPI_Reduce(
        &local_sum,        // локальное значение
        &global_sum,       // глобальный результат (только root)
        1,                 // количество элементов
        MPI_DOUBLE,        // тип данных
        MPI_SUM,           // операция суммирования
        0,                 // root-процесс
        MPI_COMM_WORLD
    );

    // =================================================
    // Окончание измерения времени
    // =================================================
    double end = MPI_Wtime();

    // =================================================
    // Вывод результатов (только root)
    // =================================================
    if (rank == 0) {
        std::cout << "Processes: " << size << "\n";
        std::cout << "Execution time: " << end - start << " seconds\n";
        std::cout << "Global sum: " << global_sum << "\n\n";
    }

    // Завершение работы MPI
    MPI_Finalize();
    return 0;
}


Overwriting task4.cpp


In [5]:
!mpic++ task4.cpp -o task4
!mpirun --allow-run-as-root --oversubscribe -np 1 ./task4
!mpirun --allow-run-as-root --oversubscribe -np 2 ./task4
!mpirun --allow-run-as-root --oversubscribe -np 4 ./task4
!mpirun --allow-run-as-root --oversubscribe -np 8 ./task4

Processes: 1
Execution time: 0.0373409 seconds
Global sum: 1e+07

Processes: 2
Execution time: 0.0711676 seconds
Global sum: 2e+07

Processes: 4
Execution time: 0.124943 seconds
Global sum: 4e+07

Processes: 8
Execution time: 0.248887 seconds
Global sum: 8e+07

